Table 2.8.1: Age and gender breakdown of young people starting treatment in 2016-17 and reported sexual exploitation

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/statistics/substance-misuse-treatment-for-young-people-statistics-2016-to-2017')
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda t: t.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['2.8.1 Sexual exploitation']

In [3]:
observations = tab.excel_ref('B6').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<C6 0.12>, <F6 413.0>, <I9 0.24>, <G10 0.21>, <I8 0.27>, <C8 0.28>, <H9 1853.0>, <D11 123.0>, <G9 0.21>, <E8 0.21>, <F9 826.0>, <B10 107.0>, <B6 66.0>, <C11 1.0>, <B7 116.0>, <I11 1.0>, <D8 26.0>, <H6 683.0>, <F7 782.0>, <I10 0.25>, <E10 0.19>, <H8 2081.0>, <D6 16.0>, <E9 0.3>, <E7 0.17>, <B9 120.0>, <G8 0.28>, <C9 0.21>, <G7 0.2>, <H11 7804.0>, <H10 1986.0>, <I7 0.15>, <C7 0.21>, <B11 565.0>, <E11 1.0>, <G6 0.1>, <B8 156.0>, <F10 834.0>, <D7 21.0>, <D10 23.0>, <H7 1201.0>, <G11 1.0>, <I6 0.09>, <F8 1094.0>, <F11 3949.0>, <C10 0.19>, <E6 0.13>, <D9 37.0>}

In [4]:
age = tab.excel_ref('A6').expand(DOWN).is_not_blank() - tab.excel_ref('A11')
age

{<A9 '16-17'>, <A8 '15-16'>, <A6 'Under 14∆'>, <A7 '14-15'>, <A13 '∆ Due to low numbers when breaking down by age and gender, age groups under 14 are combined in this table.'>, <A10 '17-18'>}

In [5]:
measuretype = tab.excel_ref('B5').expand(RIGHT).is_not_blank() 
measuretype

{<F5 'n'>, <C5 '%'>, <D5 'n'>, <G5 '%'>, <B5 'n'>, <H5 'n'>, <I5 '%'>, <E5 '%'>}

In [6]:
sex = tab.excel_ref('B4').expand(RIGHT).is_not_blank() 
sex

{<H4 'Male'>, <B4 'Female'>, <D4 'Male'>, <F4 'Female'>}

In [7]:
basis = tab.excel_ref('B3').expand(RIGHT).is_not_blank() 
basis

{<F3 'Total new presentations'>, <B3 'Sexual exploitation'>}

In [8]:
Dimensions = [
            HDim(basis,'Basis of treatment',CLOSEST,LEFT),
            HDim(age,'Clients in treatment',DIRECTLY, LEFT),
            HDim(measuretype,'Measure Type',DIRECTLY,ABOVE),
            HDim(sex,'sex',CLOSEST,LEFT),
            HDimConst('Unit','People')            
            ]

In [9]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [10]:
new_table = c1.topandas()
new_table

NoLookupError: No header found for <B11 565.0>

In [ ]:
new_table = new_table[new_table['OBS'] != 0 ]

In [ ]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [ ]:
new_table.head()

In [ ]:
new_table['Measure Type'] = new_table['Measure Type'].map(
    lambda x: {
        'n' : 'Count', 
        '%' : 'Percentage',
        }.get(x, x))

In [ ]:
new_table.tail()

In [ ]:
new_table['Clients in treatment'].fillna('All years', inplace = True)

In [ ]:
new_table.dtypes

In [ ]:
new_table['Value'] = new_table['Value'].astype(str)

In [ ]:
new_table.head(3)

In [ ]:
new_table['sex'] = new_table['sex'].map(
    lambda x: {
        'Female' : 'F', 
        'Male' : 'M',
        'Persons' : 'T'
        }.get(x, x))

In [ ]:
new_table['Clients in treatment'] = new_table['Clients in treatment'].str.rstrip('∆')

In [ ]:
new_table['Clients in treatment'] = new_table['Clients in treatment'] + '/' + new_table['sex']

In [ ]:
new_table['Clients in treatment'] =  'Ag/' + new_table['Clients in treatment']

In [ ]:
new_table['Period'] = '2016-17'
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [ ]:
new_table.tail()